<a href="https://colab.research.google.com/github/DhruvaBansal00/CV-Spring2020-Project/blob/main/baseline_BiT_M.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf

from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt

import tensorflow_hub as hub

import tensorflow_datasets as tfds


In [ ]:
# Import cifar10 data from tfds

dataset_name = 'cifar100'
ds_train, info = tfds.load(name=dataset_name, split='train', with_info=True)
ds_test = tfds.load(name=dataset_name, split='test')
num_examples = info.splits['train'].num_examples
NUM_CLASSES = 100

Shuffling and writing examples to /root/tensorflow_datasets/cifar100/3.0.2.incompleteRH8NUT/cifar100-train.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/cifar100/3.0.2.incompleteRH8NUT/cifar100-test.tfrecord


Dataset cifar100 downloaded and prepared to /root/tensorflow_datasets/cifar100/3.0.2. Subsequent calls will reuse this data.


In [ ]:
# Split into train and test sets
# We have checked that the classes are reasonably balanced.
train_split = 0.98
num_train = int(train_split * num_examples)
ds_train = ds_train.take(num_train)

DATASET_NUM_TRAIN_EXAMPLES = num_examples

In [ ]:
# Load model from TFHub into KerasLayer
model_url = "https://tfhub.dev/google/bit/m-r101x1/1"
module = hub.KerasLayer(model_url)

In [ ]:
class MyBiTModel(tf.keras.Model):
  """BiT with a new head."""

  def __init__(self, num_classes, module):
    super().__init__()

    self.num_classes = num_classes
    self.head = tf.keras.layers.Dense(num_classes, kernel_initializer='zeros')
    self.bit_model = module
  
  def call(self, images):
    # No need to cut head off since we are using feature extractor model
    bit_embedding = self.bit_model(images)
    return self.head(bit_embedding)

model = MyBiTModel(num_classes=NUM_CLASSES, module=module)

In [ ]:

# Preprocessing helper functions
RESIZE_TO = 160
CROP_TO = 128
# Create data pipelines for training and testing:
BATCH_SIZE = 512
SCHEDULE_LENGTH = 10000
SCHEDULE_LENGTH = SCHEDULE_LENGTH * 512 / BATCH_SIZE

STEPS_PER_EPOCH = 10

def cast_to_tuple(features):
  return (features['image'], features['label'])
  
def preprocess_train(features):
  # Apply random crops and horizontal flips for all tasks 
  # except those for which cropping or flipping destroys the label semantics
  # (e.g. predict orientation of an object)
  features['image'] = tf.image.random_flip_left_right(features['image'])
  features['image'] = tf.image.resize(features['image'], [RESIZE_TO, RESIZE_TO])
  features['image'] = tf.image.random_crop(features['image'], [CROP_TO, CROP_TO, 3])
  features['image'] = tf.cast(features['image'], tf.float32) / 255.0
  return features

def preprocess_test(features):
  features['image'] = tf.image.resize(features['image'], [RESIZE_TO, RESIZE_TO])
  features['image'] = tf.cast(features['image'], tf.float32) / 255.0
  return features

pipeline_train = (ds_train
                  .shuffle(200000)
                  .repeat(int(SCHEDULE_LENGTH * BATCH_SIZE / DATASET_NUM_TRAIN_EXAMPLES * STEPS_PER_EPOCH) + 1 + 50)  # repeat dataset_size / num_steps
                  .map(preprocess_train, num_parallel_calls=8)
                  .batch(BATCH_SIZE)
                  .map(cast_to_tuple)  # for keras model.fit
                  .prefetch(2))

pipeline_test = (ds_test.map(preprocess_test, num_parallel_calls=1)
                  .map(cast_to_tuple)  # for keras model.fit
                  .batch(BATCH_SIZE)
                  .prefetch(2))

In [ ]:
 SCHEDULE_BOUNDARIES = [3000, 6000, 9000]
lr = 0.003 * BATCH_SIZE / 512 
# Decay learning rate by a factor of 10 at SCHEDULE_BOUNDARIES.
lr_schedule = tf.keras.optimizers.schedules.PiecewiseConstantDecay(boundaries=SCHEDULE_BOUNDARIES, 
                                                                   values=[lr, lr*0.1, lr*0.001, lr*0.0001])
optimizer = tf.keras.optimizers.SGD(learning_rate=lr_schedule, momentum=0.9)

loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

In [ ]:
model.compile(optimizer=optimizer,
              loss=loss_fn,
              metrics=['accuracy'])

# Fine-tune model
history = model.fit(
    pipeline_train,
    batch_size=BATCH_SIZE,
    steps_per_epoch=STEPS_PER_EPOCH,
    epochs= int(SCHEDULE_LENGTH / STEPS_PER_EPOCH), 
    validation_data=pipeline_test  
)

Epoch 1/1000
10/10 [==============================] - 48s 5s/step - loss: 3.9724 - accuracy: 0.2457 - val_loss: 2.5944 - val_accuracy: 0.5508
Epoch 2/1000
10/10 [==============================] - 45s 4s/step - loss: 2.5092 - accuracy: 0.4873 - val_loss: 1.3599 - val_accuracy: 0.6786
Epoch 3/1000
10/10 [==============================] - 45s 4s/step - loss: 1.5161 - accuracy: 0.6582 - val_loss: 1.0721 - val_accuracy: 0.7121
Epoch 4/1000
10/10 [==============================] - 45s 4s/step - loss: 1.7773 - accuracy: 0.6186 - val_loss: 0.9853 - val_accuracy: 0.7203
Epoch 5/1000
10/10 [==============================] - 46s 5s/step - loss: 1.3143 - accuracy: 0.6893 - val_loss: 0.9454 - val_accuracy: 0.7299
Epoch 6/1000
10/10 [==============================] - 45s 4s/step - loss: 1.3868 - accuracy: 0.6814 - val_loss: 0.9099 - val_accuracy: 0.7375
Epoch 7/1000
10/10 [==============================] - 45s 5s/step - loss: 1.2370 - accuracy: 0.7160 - val_loss: 0.9009 - val_accuracy: 0.7464
Epoch 

KeyboardInterrupt: ignored